In [1]:
import matplotlib.pyplot as plt
import seaborn as sns

#NLP libraries
import nltk
from gensim.models import Doc2Vec
import gensim
from gensim.models.doc2vec import TaggedDocument
import urllib

#Machine learning libraries
from sklearn import utils
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.utils import shuffle

#Helper libraries
import multiprocessing
import numpy as np
import pandas as pd
import math
from bs4 import BeautifulSoup
import re
import os


In [2]:
#Training data
path=".."
train_data_0=pd.read_csv(path+"/scraped_data/3000_rationalwiki_training_articles.csv")
train_data_0["text_content"]=train_data_0["text_content"].replace({r"\r\n" : " ", "\n": " "},regex=True)
train_data_0["bias"]=0

train_data_2=pd.read_csv(path+"/scraped_data/3000_infogalactic_training_articles.csv")
train_data_2["text_content"]=train_data_2["text_content"].replace({r"\r\n" : " ", "\n": " "},regex=True)
train_data_2["bias"]=1

train_data=pd.concat([train_data_0,train_data_2], ignore_index=True)



#Pred data
text_file_dir=path+"/data/wikispeedia_articles_plaintext/plaintext_articles/"
file_data=[]

for filename in os.listdir(text_file_dir):
    with open(os.path.join(text_file_dir, filename), "r") as file:
        content = file.read()

        # Split the content into lines to remove the header
    lines = content.split('\n')

    #Removing the header (line 0)
    if lines:
        lines.pop(0)

    # Making it an array and removing all \n
    content = '\n'.join(lines)
    content = content.replace("\n", " ")

    filename_fin=filename[:-4]
    new_file_content= {"text_content": content, "title": urllib.parse.unquote(filename_fin , encoding = 'utf-8')}

    file_data.append(new_file_content)

pred_data = pd.DataFrame(file_data)


In [6]:
def clean(text):  #Removing unecessary punctuation and all lower case.
    text = re.sub(r'\|\|\|', r' ', text)
    text = text.replace('„','')
    text = text.replace('“','')
    text = text.replace('"','')
    text = text.replace('\'','')
    text = text.replace('-','')

    for i in range(692): #removing annotations
        text= text.replace(f"[{i}]","")
        text= text.replace(f"[note {i}]", "")

    text = text.lower()
    return text

In [7]:
train_data['text_content'] = train_data['text_content'].apply(clean)
pred_data['text_content'] = pred_data['text_content'].apply(clean)

In [8]:
train_data.to_csv("./cleaned_train.csv", index=False)
pred_data.to_csv("./cleaned_pred.csv", index=False)

In [9]:
train_data["text_content"][0]

'electroconvulsive therapy electroconvulsive therapy  ( ect ), also known by the nowdisfavored terms  electroshock therapy  or  shock treatment , is one of the most controversial aspects of 20 th  and 21 st  century  psychiatry . the idea that strapping a distressed patient to a bed and artificially inducing seizures via shocks to the brain can have a worthwhile, safe, and therapeutic effect for mood is as ridiculous as it sounds. ect has had a rollercoaster ride in reputation: once viewed as a  miracle  cure, it became (along with the equally maligned practice of  lobotomy ) a symbol of the evils and undeniable ethical failures of psychiatry in the 1960s to today. ect has been applied in clinical practice for over 70 years,     and today is mainly used on severe and vulnerable depressives, such as those who did not respond to drug interventions and could not or would not escape psychiatry altogether. ect is performed involuntarily on many such patients, including today,     against un